# Data Loading and Preprocessing 
Connect to Colab instance to Google Drive to access dataset from shared Drive Folder


In [ ]:
!pip install cupy-cuda102

In [ ]:
!pip install numpy==1.19

In [ ]:
!pip install pywt

In [ ]:
from numba import cuda

In [1]:
import scipy.io
import scipy.signal
import seaborn as sns
import matplotlib.pyplot as plt
import cupy as cp
import numpy as np
import cusignal
import os
#import pywt
import torch
from torch.utils.dlpack import to_dlpack
from torch.utils.dlpack import from_dlpack
import pandas as pd
cpsc_dir = 'CPSC'
ptb_dir = 'PTB'
ptb_xl_dir = 'PTB-XL'
stpet_dir = 'StPet'
georgia_dir = 'Training_E'
china_dir = 'Training_2'

In [2]:
codes = {
    270492004:0,
    164889003:1,
    426627000:2,
    713427006:3,
    445118002:4,
    39732003:5,
    164909002:6,
    251146004:7,
    698252002:8,
    10370003:9,
    284470004:10,
    427172004:11,
    164947007:12,
    111975006:13,
    164917005:14,
    47665007:15,
    59118001:16,
    427393009:17,
    59118001:18,
    427393009:19,
    426177001:20,
    426783006:21,
    427084000:22,
    63593006:23,
    164934002:24,
    59931005:25,
    17338001:26,
}
print(codes.keys())

dict_keys([270492004, 164889003, 426627000, 713427006, 445118002, 39732003, 164909002, 251146004, 698252002, 10370003, 284470004, 427172004, 164947007, 111975006, 164917005, 47665007, 59118001, 427393009, 426177001, 426783006, 427084000, 63593006, 164934002, 59931005, 17338001])


Resample PTB to 500 HZ

In [6]:
ptb = torch.zeros(size=(96,12,7500)).cuda()
labels = torch.zeros(size=(96,1)).cuda()
print(ptb.device)
#Finite Impulse Response 
bandpass = cusignal.firwin(250, np.arange(3,46), fs=500)
plt.rcParams['figure.figsize'] = (30,20)
af = 0 
afl = 0
st = 0
sr = 0
cnt = 0
dx=''
for root, dirs, files in os.walk(ptb_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            if cnt % 10 == 0:
                print(cnt)
            if dx == '164889003':
                af+=1
            elif dx == '164890007':
                afl+=1
            elif dx == '426783006':
                sr+=1
            elif dx == '427084000':
                st+=1
            else:
                assert dx == ''
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            for i in range(12):
                temp = cusignal.resample(signal[i], 500*len(signal[i])//1000)
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(temp, bandpass, mode = 'same')
                #plot, ax = plt.subplots()
                #ax.plot(csig.get())

                #print(csig)
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = csig.max()
                mi = csig.min()
                for k in range(len(csig)):
                    #print(float(csig[k]))
                    assert csig[k] <= ma and csig[k] >= mi
                    ptb[cnt-1][i][k] = (csig[k] - mi)/(ma-mi)
                    assert ptb[cnt-1][i][k] >= 0 and ptb[cnt-1][i][k] <= 1
                labels[cnt-1][0] = codes[int(dx)]
print(cnt)
print("AF:",af)
print("AFL:", afl)
print("SR:", sr)
print("STach:", st)
torch.save(ptb, 'ptb.pt')
torch.save(labels, 'ptblabels.pt')
del ptb
del labels

cuda:0
10
20
30
40
50
60
70
80
90
96
AF: 15
AFL: 0
SR: 80
STach: 1


In [ ]:
import torch.nn as nn 
import torch.nn.functional as F 
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 42):
        super.__init__()
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x*math.sqrt(self.d_model)
        seq_len = x.size(1)
        x = x + Variable(self.pe[:, :seq_len], requires_grad=False).cuda()
        return x
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super.__init__()
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
        
        
class convNet(nn.Module):
    def __init__(self):
        super(convNet, self).__init__()
        self.conv1 = nn.Conv1d(12, 64, 14, 3)
        self.conv2 = nn.Conv1d(64, 128, 14, 2)
        self.conv3 = nn.Conv1d(128, 128, 10, 1)
        #self.fc1 = nn.Linear(5376, 200)
        self.fc2 = nn.Linear(200, 27)
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), 3)
        x = F.max_pool1d(F.relu(self.conv2(x)), 3)
        x = F.max_pool1d(F.relu(self.conv3(x)), 3)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x


In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import KFold
labels = torch.squeeze(labels)
labels = labels.long()
dataset = data_utils.TensorDataset(ptb, labels)
#dataset_loader = data_utils.DataLoader(dataset, batch_size=20, shuffle=True)
kfold = KFold(n_splits = 3, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')
    print('----------------------')
    train_subsampler = data_utils.SubsetRandomSampler(train_ids)
    test_subsampler = data_utils.SubsetRandomSampler(test_ids)
    trainloader = data_utils.DataLoader(
        dataset,
        batch_size=10, sampler=train_subsampler
    )
    testloader = data_utils.DataLoader(
        dataset,
        batch_size=10, sampler=test_subsampler
    )
    model = convNet().cuda()
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_function = nn.CrossEntropyLoss()
    for epoch in range(0, 50):
        print(f'Starting epoch {epoch+1}')
        current_loss = 0.0
        for i,data in enumerate(trainloader,0):
            inputs, targets = data
            optimizer.zero_grad()
            outputs=model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            current_loss += loss.item()
        print(f'Loss after epoch: {current_loss}')
    correct, total = 0,0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs, targets = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data,1)
            total += targets.size(0)
            correct += (predicted==targets).sum().item()
        print('Accuracy for fold {}: {}'.format(fold+1, 100 * correct/total))
            

In [ ]:
torch.save(model,'./model.h5')

In [ ]:
model = torch.load('../input/ptbcnn/ptbBasicCNN.h5')

In [ ]:
for parameter in model.parameters():
    if parameter.requires_grad:
        print(parameter.numel())

In [9]:
stpet = torch.zeros(size=(33,12,7500)).cuda()
labels = torch.zeros(size=(33,1)).cuda()
cnt = 0
for root, dirs, files in os.walk(stpet_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            for i in range(12):
                temp = cusignal.resample(signal[i], 500*len(signal[i])//1000)
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(temp, bandpass, mode = 'same')
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = max(csig)
                mi = min(csig)
                for k in range(len(csig)):
                    stpet[cnt-1][i][k] = (csig[k] - mi)/(ma-mi)
                    assert stpet[cnt-1][i][k] >= 0 and stpet[cnt-1][i][k] <= 1
                labels[cnt-1][0] = codes[int(dx)]

print(cnt)
torch.save(stpet, 'stpet.pt')
torch.save(labels, 'stpetlabels.pt')
del stpet
del labels

33


In [ ]:
ptbxl = torch.zeros(size=(11554,12,7500)).cuda()
labels = torch.zeros(size=(11554, 1)).cuda()
cnt = 0
bandpass = cusignal.firwin(30, 45, fs=500)
for root, dirs, files in os.walk(ptb_xl_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
       
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            #print(cnt)
            if cnt >= 11554:
                continue
            if cnt % 10 == 0:
                print(cnt)
            
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            
            for i in range(12):
                temp = signal[i]
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(temp, bandpass, mode = 'same')
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = max(csig)
                mi = min(csig)
                #rint(ma)
                #rint(mi)
                for k in range(len(csig)):
                    ptbxl[cnt][i][k] = (csig[k] - mi)/(ma-mi)
                    #rint(ptbxl[cnt][i][k])
                    assert ptbxl[cnt][i][k] >=0 and ptbxl[cnt][i][k] <=1
                labels[cnt][0] = codes[int(dx)]

             
print(cnt)
torch.save(ptbxl, 'ptbxl2.pt')
torch.save(labels, 'ptbxllabels2.pt')
del ptbxl
del labels

In [ ]:
georgia = torch.zeros(size=(9348,12,7500)).cuda()
bandpass = cusignal.firwin(250, np.arange(3,46), fs=500)
labels = torch.zeros(size=(9348,1)).cuda()
cnt = 0
for root, dirs, files in os.walk(georgia_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            #print(cnt)
            if cnt % 10 == 0:
                print(cnt)
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            
            for i in range(12):
                temp = signal[i]
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(temp, bandpass, mode = 'same')
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = max(csig)
                mi = min(csig)
                for k in range(len(csig)):
                    georgia[cnt-1][i][k] = (csig[k] - mi)/(ma-mi)
                    assert georgia[cnt-1][i][k] >= 0 and georgia[cnt-1][i][k] <=1
                labels[cnt-1][0] = codes[int(dx)]
            
print(cnt)
torch.save(georgia, 'georgia.pt')
torch.save(labels, 'georgialabels.pt')
del georgia
del labels

10
20
30


In [ ]:
cpsc = torch.zeros(size=(5279,12,7500)).cuda()
labels = torch.zeros(size=(5279,1)).cuda()
cnt = 0
for root, dirs, files in os.walk(cpsc_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            #print(cnt)
            
            if cnt % 10 == 0:
                print(cnt)
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            for i in range(12):
                temp = signal[i]
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(cpsc_signals[i][j], bandpass, mode = 'same')
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = max(csig)
                mi = min(csig)
                for k in range(len(csig)):
                    cpsc[cnt-1][i][k] = (csig[k] - mi)/(ma-mi)
                    assert cpsc[cnt][i][k] >= 0 and cpsc[cnt][i][k] <=1
                labels[cnt-1][0] = codes[int(dx)]

print(cnt)
torch.save(cpsc, 'cpsc.pt')
torch.save(labels, 'cpsclabels.pt')
del cpsc
del labels


In [ ]:
china = torch.zeros(size=(1203,12,7500)).cuda()
labels = torch.zeros(size=(1203,1)).cuda()
cnt = 0
for root, dirs, files in os.walk(china_dir, topdown=False):
  for name in files:
        path = os.path.join(root, name)
        #print(cnt)
        if path[-3:] == 'hea':
            dx = ''
            cont = False
            object_file = open(path, 'r')
            line = object_file.readline()
            line = line.rstrip('\n')
            count = 0
            diag = ''
            while count <= 100:
                if line == '' :
                    count += 5
                else:
                #print(line)
                    line = object_file.readline()
                    line = line.rstrip('\n')
                if line[1:3] == 'Dx':
                    #print(cnt)
                    #print(line)
                    #print(line[5:])
                    diag = line[5:]
                prev = 0
                for i in range(len(diag)):
                    if diag[i] == ',':
                        if int(diag[prev:i]) in codes.keys():
                            dx = diag[prev:i]
                        prev = i+1
                    if i == len(diag)-1:
                        if int(diag[prev:]) in codes.keys():
                            dx = diag[prev:]
            count += 1
            object_file.close()
        if dx == '':
            cont = True
        else:
            cont = False
        
        if cont == False and path[-3:] == 'mat':
            cnt+=1
            #print(cnt)
            
            if cnt % 10 == 0:
                print(cnt)
            signal = scipy.io.loadmat(path)['val']
            signal = cp.array(signal)
            for i in range(12):
                temp = signal[i]
                if temp.shape[0] > 7500:
                    temp = temp[0:7500]
                temp = cp.array(temp)
                csig = cusignal.convolve(temp, bandpass, mode = 'same')
                csig = torch.as_tensor(csig, dtype=torch.float).cuda()
                ma = max(csig)
                mi = min(csig)
                for k in range(len(csig)):
                    china[cnt-1][i][k] = (csig[k] - mi)/(ma-mi) 
                    assert china[cnt-1][i][k] >= 0 and china[cnt-1][i][k] <= 1
                labels[cnt-1][0] = codes[int(dx)]

            
print(cnt)
torch.save(china, 'china.pt')
torch.save(labels, 'chinalabels.pt')
del china
del labels